<a href="https://colab.research.google.com/github/Pedrobertuola/Mercado-Financeiro/blob/main/PriceCryptoPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
import datetime as dt

In [ ]:
crypto_currency = 'TRX'
against_currency = 'USD'

In [ ]:
start = dt.datetime(2018,1,1)
end= dt.datetime.now()
df = web.DataReader(f'{crypto_currency}-{against_currency}', 'yahoo', start, end)


In [ ]:


#Show the data
df = df.dropna()

In [ ]:
df = df[['Close']]
df

,Close
Date,
2018-01-01,0.051695
2018-01-02,0.078682
2018-01-03,0.094703
2018-01-04,0.207974
2018-01-05,0.220555
...,...
2021-05-12,0.121473
2021-05-13,0.124776
2021-05-14,0.120691


In [ ]:
#Create a variable to store the number of days into the future that we want to predict
prediction_days =1
#Create a new column called Prediction
df['Prediction'] = df[['Close']].shift(-prediction_days)

#show the datas
df

,Close,Prediction
Date,,
2018-01-01,0.051695,0.078682
2018-01-02,0.078682,0.094703
2018-01-03,0.094703,0.207974
2018-01-04,0.207974,0.220555
2018-01-05,0.220555,0.174797
...,...,...
2021-05-12,0.121473,0.124776
2021-05-13,0.124776,0.120691
2021-05-14,0.120691,0.119506


In [ ]:
#Create the independent data set (X)
X = np.array(df.drop(['Prediction'], 1))
#Remove the last n+1 rows of data where n is the prediction_days
X = X[:len(df) -prediction_days -1]
#Print the data
print(X)

[[0.051695]
 [0.078682]
 [0.094703]
 ...
 [0.121473]
 [0.124776]
 [0.120691]]


In [ ]:
#Create a dependent data set (y)
y = np.array(df['Prediction'])
#Get all of the y values except the last n+1 rows
y =  y[:-prediction_days -1 ]
#Show the data
print(y)

[0.078682 0.094703 0.207974 ... 0.124776 0.120691 0.119506]


In [ ]:
#Split the data into 80% training data set and a 20% testing data set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
#Use the randomForestRegressor for the model
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(n_estimators = 2, random_state =587)
forest.fit(x_train, y_train)
print(forest.score(x_test, y_test))

0.8957766563085383


In [ ]:
#Show how close the predicted values and the actual values are
prediction = forest.predict(x_test)
#Print the predicted test values
print(prediction)
print()
#Print the actual values
print(y_test)

[0.01724625 0.027819   0.048028   0.02228223 0.01011967 0.02263596
 0.01175628 0.06103    0.031176   0.01724625 0.1211215  0.01334525
 0.01794297 0.01547923 0.025082   0.013623   0.017256   0.02322667
 0.152484   0.0649     0.010365   0.01443622 0.0443623  0.0145635
 0.0243872  0.027062   0.03148642 0.01468865 0.02429933 0.0366475
 0.02000567 0.02327916 0.02583088 0.014283   0.10895542 0.0201248
 0.0138905  0.0303922  0.030423   0.069992   0.02722    0.1227445
 0.04872067 0.02633425 0.03046425 0.038423   0.02482    0.03043525
 0.02653133 0.0380545  0.02585209 0.0654385  0.04474825 0.036256
 0.0654385  0.0161455  0.02808357 0.0130119  0.043075   0.03497264
 0.018179   0.0649     0.0178825  0.01794297 0.01466172 0.025739
 0.0145635  0.0270115  0.01333125 0.02012649 0.02012649 0.024138
 0.03758075 0.01763567 0.0155865  0.016477   0.01929521 0.053793
 0.014283   0.02403852 0.02172286 0.10908    0.01744017 0.0243872
 0.02695459 0.016785   0.01929521 0.016785   0.01475708 0.02115617
 0.02322

In [ ]:
#Get the validation data for the model
#Create a variable to store all of the rows in the data set except the last n rows
temp_df = df[:-prediction_days]
#Create a variable to store the independent price value
x_val = temp_df.tail(1)['Close'][0]
#Show the data
print(x_val)

0.11950600147247314


In [ ]:
prediction = forest.predict([[x_val]])
#Print the price of Dogecoin for the next n days

print(f'The price of {crypto_currency} in ', prediction_days, 'day(s) is predicted to be', prediction)
print('The actual price was', temp_df.tail(1)['Prediction'][0])

The price of TRX in  1 day(s) is predicted to be [0.119506]
The actual price was 0.11474662274122238


In [ ]:
ganho= float(((prediction-temp_df.tail(1)['Prediction'][0])/temp_df.tail(1)['Prediction'][0])*100)
print('Ganho previsto com operação', ganho, '%')

Ganho previsto com operação 4.147728811142579 %
